# Handling Missing Values in Pandas

This notebook is a step-by-step tutorial for **beginners in data analytics with Python**. You will learn what missing values are, how to detect them, and several practical ways to handle them using the pandas library.

We will use a small sensor dataset called `sensor_log.csv` as a running example.

### Learning goals
By the end of this notebook you should be able to:
- Explain what a missing value is and why it matters.
- Load data into a pandas DataFrame and check for missing values.
- Summarise how many missing values each column has.
- Decide when to drop rows or columns that contain missing values.
- Fill (impute) missing values using simple strategies such as constants, mean, median, and forward or backward fill.
- Understand the advantages and disadvantages of each approach.


## 0. Setup

We start by importing the main Python libraries we will use:

- `pandas` for working with tabular data (tables).
- `numpy` for working with numeric values such as the special `NaN` value that represents "Not a Number".


In [9]:
import pandas as pd
import numpy as np

# Optional: show the pandas version so students know which version is used
print('pandas version:', pd.__version__)


pandas version: 2.3.3


## 1. Loading the dataset and taking a first look

Our example dataset `sensor_log.csv` contains readings from a temperature and humidity sensor. Each row is one measurement.

Typical steps when you first load a dataset are:
- Look at the first few rows with `head`.
- Check the shape (how many rows and columns).
- Look at basic information about each column with `info`.


In [10]:
# Read the CSV file into a DataFrame
df = pd.read_csv('sensor_log.csv')

# Look at the first 5 rows
df.head()


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
2,2025-10-01 08:00:20,24.6,55.1,NaN
3,2025-10-01 08:00:30,NaN,54.9,3.68
4,2025-10-01 08:01:00,24.9,54.8,3.68


In [11]:
# How many rows and columns does the dataset have?
print('Number of rows:', df.shape[0])
print('Number of columns:', df.shape[1])

# General information about the DataFrame, including data types and non-null counts
df.info()


Number of rows: 10
Number of columns: 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   timestamp      10 non-null     object 
 1   temperature_c  8 non-null      float64
 2   humidity_pct   9 non-null      float64
 3   voltage_v      9 non-null      float64
dtypes: float64(3), object(1)
memory usage: 448.0+ bytes


### What is a missing value?

In real-world data, we often do not have every value for every row. For example:
- A sensor might temporarily fail to record a value.
- A user might skip a question in a survey.
- A value might be invalid or lost during data collection.

In pandas, missing numeric values are usually represented as `NaN` (Not a Number). In some datasets, missing values may also be shown as special codes such as -999 or the string 'NA'. We can convert those to proper missing values if needed.


## 2. Detecting missing values

The first step in handling missing data is **finding out where the missing values are**.

Useful pandas functions:
- `isna()` or `isnull()` return `True` where a value is missing.
- `notna()` or `notnull()` return `True` where a value is present.

We rarely look at the full `True`/`False` table. Instead, we usually sum up how many missing values are in each column.


In [12]:
# A quick look at where values are missing (True means missing)
df.isna().head()


,timestamp,temperature_c,humidity_pct,voltage_v
0,False,False,False,False
1,False,False,False,False
2,False,False,False,True
3,False,True,False,False
4,False,False,False,False


In [13]:
# Count how many missing values are in each column
df.isna().sum()


timestamp        0
temperature_c    2
humidity_pct     1
voltage_v        1
dtype: int64

In [14]:
# Calculate the percentage of missing values in each column
missing_percent = df.isna().mean() * 100
missing_percent.round(2)


timestamp         0.0
temperature_c    20.0
humidity_pct     10.0
voltage_v        10.0
dtype: float64

### Exercise 1 (for students)

1. Create a new Series or DataFrame that shows only the rows where `temperature_c` is missing.
2. Do the same for `humidity_pct`.
3. Which column in this dataset has the highest percentage of missing values?


## 3. Strategy 1: Dropping missing values

The simplest strategy is to **remove** rows or columns that contain missing values.

- `dropna()` by default drops any row that has at least one missing value.
- You can also drop columns instead of rows (by using `axis=1` or `axis='columns'`).

This can be safe if:
- Only a small number of rows are missing.
- The rows are not systematically different from the others.

It can be dangerous if you lose a lot of data, or if the missingness is not random.


In [15]:
# Drop any rows that contain at least one missing value
df_drop_rows = df.dropna()

print('Original shape:', df.shape)
print('After dropping rows with any missing values:', df_drop_rows.shape)
df_drop_rows.head()


Original shape: (10, 4)
After dropping rows with any missing values: (6, 4)


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
4,2025-10-01 08:01:00,24.9,54.8,3.68
6,2025-10-01 08:03:00,25.3,54.7,3.67
7,2025-10-01 08:05:30,25.5,54.9,3.65


In [17]:
# Drop columns that contain any missing values
df_drop_cols = df.dropna(axis='columns')

print('Columns before:', df.columns.tolist())
print('Columns after dropping any column with missing values:', df_drop_cols.columns.tolist())
df_drop_cols.head()


Columns before: ['timestamp', 'temperature_c', 'humidity_pct', 'voltage_v']
Columns after dropping any column with missing values: ['timestamp']


,timestamp
0,2025-10-01 08:00:00
1,2025-10-01 08:00:10
2,2025-10-01 08:00:20
3,2025-10-01 08:00:30
4,2025-10-01 08:01:00


### When is dropping okay?

Dropping rows or columns with missing values may be acceptable when:
- The percentage of missing values is very small.
- You are sure that losing those rows will not bias your analysis.

However, if you drop too much data, your results may no longer represent the real situation. In those cases, filling the missing values may be better.


## 4. Strategy 2: Filling missing values with simple rules

Instead of removing data, we can **fill in** missing values with reasonable guesses. This process is called *imputation*.

Common simple strategies:
- Fill with a fixed constant (for example 0 or a special code).
- Fill numeric columns with the column mean or median.
- For categorical columns (for example city or category), fill with the most frequent value (the mode).

In pandas we usually use the `fillna` function for this.


In [18]:
# Example: fill missing voltage values with a constant
df_constant = df.copy()

df_constant['voltage_v'] = df_constant['voltage_v'].fillna(0)

df_constant.head()


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
2,2025-10-01 08:00:20,24.6,55.1,0.00
3,2025-10-01 08:00:30,NaN,54.9,3.68
4,2025-10-01 08:01:00,24.9,54.8,3.68


In [19]:
# Fill all numeric columns with their column mean
df_mean = df.copy()
numeric_cols = df_mean.select_dtypes(include='number').columns

for col in numeric_cols:
    col_mean = df_mean[col].mean()
    df_mean[col] = df_mean[col].fillna(col_mean)

df_mean.head()


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.500,55.2,3.700000
1,2025-10-01 08:00:10,24.700,55.0,3.690000
2,2025-10-01 08:00:20,24.600,55.1,3.667778
3,2025-10-01 08:00:30,25.075,54.9,3.680000
4,2025-10-01 08:01:00,24.900,54.8,3.680000


In [20]:
# Check that there are no missing values left in the numeric columns
df_mean[numeric_cols].isna().sum()


temperature_c    0
humidity_pct     0
voltage_v        0
dtype: int64

In [21]:
# (Optional) Example with a small categorical column
example = pd.DataFrame({
    'city': ['Accra', 'Accra', 'Kumasi', np.nan, 'Accra'],
    'temperature_c': [30, 31, 29, 28, np.nan]
})

print('Original example DataFrame:')
display(example)

# Fill missing city with the most frequent city (the mode)
city_mode = example['city'].mode()[0]
example['city'] = example['city'].fillna(city_mode)

# Fill missing temperature with the median
temp_median = example['temperature_c'].median()
example['temperature_c'] = example['temperature_c'].fillna(temp_median)

print('After filling missing values:')
display(example)


Original example DataFrame:


,city,temperature_c
0,Accra,30.0
1,Accra,31.0
2,Kumasi,29.0
3,NaN,28.0
4,Accra,NaN


After filling missing values:


,city,temperature_c
0,Accra,30.0
1,Accra,31.0
2,Kumasi,29.0
3,Accra,28.0
4,Accra,29.5


### Exercise 2 (for students)

1. Create a copy of `df` called `df_median`.
2. For each numeric column, fill the missing values with the column median.
3. Compare the results of mean-based imputation (`df_mean`) and median-based imputation (`df_median`). Which do you think is more robust to extreme values (outliers)?


## 5. Strategy 3: Time series methods (forward fill, backward fill, interpolation)

For time series data (data ordered by time), it is often reasonable to use information from nearby timestamps to fill in missing values.

Three common methods are:
- **Forward fill (ffill)**: copy the last known value forward to fill the gap.
- **Backward fill (bfill)**: copy the next known value backward.
- **Interpolation**: smoothly estimate missing values between known points.

We will first make sure that the `timestamp` column is treated as a proper datetime type and set as the index.


In [22]:
# Prepare a time-indexed version of the data
df_ts = df.copy()
df_ts['timestamp'] = pd.to_datetime(df_ts['timestamp'])
df_ts = df_ts.set_index('timestamp')

df_ts.head()


,temperature_c,humidity_pct,voltage_v
timestamp,,,
2025-10-01 08:00:00,24.5,55.2,3.70
2025-10-01 08:00:10,24.7,55.0,3.69
2025-10-01 08:00:20,24.6,55.1,NaN
2025-10-01 08:00:30,NaN,54.9,3.68
2025-10-01 08:01:00,24.9,54.8,3.68


In [23]:
# Forward fill: each missing value takes the last known value above it
df_ffill = df_ts.ffill()

df_ffill.head()


,temperature_c,humidity_pct,voltage_v
timestamp,,,
2025-10-01 08:00:00,24.5,55.2,3.70
2025-10-01 08:00:10,24.7,55.0,3.69
2025-10-01 08:00:20,24.6,55.1,3.69
2025-10-01 08:00:30,24.6,54.9,3.68
2025-10-01 08:01:00,24.9,54.8,3.68


In [24]:
# Backward fill: each missing value takes the next known value below it
df_bfill = df_ts.bfill()

df_bfill.head()


,temperature_c,humidity_pct,voltage_v
timestamp,,,
2025-10-01 08:00:00,24.5,55.2,3.70
2025-10-01 08:00:10,24.7,55.0,3.69
2025-10-01 08:00:20,24.6,55.1,3.68
2025-10-01 08:00:30,24.9,54.9,3.68
2025-10-01 08:01:00,24.9,54.8,3.68


In [25]:
# Interpolate numeric values over time
# Here we use method='time' to respect the time index
df_interp = df_ts.interpolate(method='time')

df_interp.head()


,temperature_c,humidity_pct,voltage_v
timestamp,,,
2025-10-01 08:00:00,24.500,55.2,3.700
2025-10-01 08:00:10,24.700,55.0,3.690
2025-10-01 08:00:20,24.600,55.1,3.685
2025-10-01 08:00:30,24.675,54.9,3.680
2025-10-01 08:01:00,24.900,54.8,3.680


### When are these time series methods reasonable?

- Forward fill is common for slowly changing signals, such as temperature or account balance.
- Backward fill can be useful when you know future values are valid approximations for the recent past.
- Interpolation is suitable when you expect smooth changes over time.

However, none of these methods is perfect. Always think about the meaning of the data and whether the method makes sense in your context.


### Exercise 3 (for students)

1. Create three new DataFrames from `df_ts`: one using forward fill, one using backward fill, and one using interpolation.
2. For a small time range, compare the values side by side.
3. Discuss with a partner: which method seems most reasonable for this sensor data and why?


## 6. Putting it all together: choosing a strategy

There is no single "best" way to handle missing values. The right choice depends on:

- **How much data is missing**: a few values or a large portion of the dataset.
- **Why the data is missing**: at random, due to sensor failure, due to human choices, and so on.
- **How the data will be used**: simple descriptive statistics, predictive models, control systems, etc.

Typical workflow:
1. Explore the data and understand where and how much is missing.
2. Try simple strategies such as dropping rows or filling with mean/median for numeric values.
3. For time series, consider forward fill, backward fill, or interpolation.
4. Check how your results change when you use different strategies.

As you advance in your studies, you will learn more advanced imputation methods (for example, using machine learning models to predict missing values).


## 7. Final practice (mini project)

As a final exercise, work through the following steps on your own copy of the dataset:

1. Load `sensor_log.csv` into a new DataFrame.
2. Summarise missing values per column (counts and percentages).
3. Decide, with justification, which columns or rows (if any) you would drop.
4. Choose and apply an imputation strategy for the remaining missing values (for example, mean/median or forward fill).
5. Compare key summary statistics (mean, min, max) before and after imputation.
6. Write a short paragraph explaining which decisions you made and why they are reasonable for this dataset.

This type of reasoning is a crucial skill for any data analyst.


## ✏️ Exercise 1 – Identifying Missing Values
# 1. Rows where temperature is missing
temp_missing = df[df["temperature_c"].isna()]
temp_missing

# 2. Rows where humidity is missing
humidity_missing = df[df["humidity_pct"].isna()]
humidity_missing

# 3. Which column has the highest missing percentage?
missing_percentage = df.isna().mean() * 100
missing_percentage


In [26]:
## Rows where temperature_c is missing

temp_missing = df[df["temperature_c"].isna()]
temp_missing


,timestamp,temperature_c,humidity_pct,voltage_v
3,2025-10-01 08:00:30,NaN,54.9,3.68
8,2025-10-01 08:08:00,NaN,55.0,3.64


In [27]:
## Rows where humidity_pct is missing

humidity_missing = df[df["humidity_pct"].isna()]
humidity_missing


,timestamp,temperature_c,humidity_pct,voltage_v
5,2025-10-01 08:02:15,25.1,NaN,3.67


In [28]:
## Which column has the highest percentage of missing values?

missing_percentage = df.isna().mean() * 100
missing_percentage


timestamp         0.0
temperature_c    20.0
humidity_pct     10.0
voltage_v        10.0
dtype: float64

From the missing_percentage result, I can see that temperature_c has the highest missing values. That means this variable will need more careful handling when cleaning.

## EXERCISE 2 – Median Imputation

In [29]:
## Create copy

df_median = df.copy()


In [31]:
## Fill missing values with MEDIAN

for col in ["temperature_c", "humidity_pct", "voltage_v"]:
    df_median[col] = df_median[col].fillna(df_median[col].median())


In [32]:
## Compare MEAN vs MEDIAN imputation

print("Mean Imputation (df_mean):")
display(df_mean.head())

print("Median Imputation (df_median):")
display(df_median.head())


Mean Imputation (df_mean):


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.500,55.2,3.700000
1,2025-10-01 08:00:10,24.700,55.0,3.690000
2,2025-10-01 08:00:20,24.600,55.1,3.667778
3,2025-10-01 08:00:30,25.075,54.9,3.680000
4,2025-10-01 08:01:00,24.900,54.8,3.680000


Median Imputation (df_median):


,timestamp,temperature_c,humidity_pct,voltage_v
0,2025-10-01 08:00:00,24.5,55.2,3.70
1,2025-10-01 08:00:10,24.7,55.0,3.69
2,2025-10-01 08:00:20,24.6,55.1,3.67
3,2025-10-01 08:00:30,25.0,54.9,3.68
4,2025-10-01 08:01:00,24.9,54.8,3.68


Median is more robust to extreme values because it ignores outliers.
If there is a spike or drop in sensor values, median filling will not be affected.
Mean filling can distort natural behavior when outliers are present.

## EXERCISE 3 – Compare FILLING METHODS

In [33]:
## Compare a small range of rows

df_compare = pd.DataFrame({
    "original": df["temperature_c"],
    "forward_fill": df_ffill["temperature_c"],
    "backward_fill": df_bfill["temperature_c"],
    "interpolation": df_interp["temperature_c"]
})

df_compare.head(10)   # Compare first 10 rows


,original,forward_fill,backward_fill,interpolation
0,24.5,NaN,NaN,NaN
1,24.7,NaN,NaN,NaN
2,24.6,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,24.9,NaN,NaN,NaN
5,25.1,NaN,NaN,NaN
6,25.3,NaN,NaN,NaN
7,25.5,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN
9,26.0,NaN,NaN,NaN


Interpolation looks the most realistic because the values change smoothly.
Forward and backward fill repeat previous/future values — this may be okay for small gaps.
Mean/median remove natural variation, so they may not be ideal for sensor data.

## Final Mini Project – Missing Value Decision Making


In [34]:
# 1. Load clean copy
df2 = pd.read_csv("sensor_log.csv")

# 2. Missing values summary
missing_counts = df2.isna().sum()
missing_percent = df2.isna().mean() * 100

display(missing_counts)
display(missing_percent)

# 3. Decide if any rows/columns should be DROPPED
# ➤ Example: Drop rows with all values missing
df2 = df2.dropna(how="all")

# 4. Apply imputation (choose your preferred method)
df2_imputed = df2.interpolate()    # <-- Or use median/mean instead

# 5. Compare statistics before & after
before = df2.describe().T
after = df2_imputed.describe().T

before, after


timestamp        0
temperature_c    2
humidity_pct     1
voltage_v        1
dtype: int64

timestamp         0.0
temperature_c    20.0
humidity_pct     10.0
voltage_v        10.0
dtype: float64

C:\Users\user\AppData\Local\Temp\ipykernel_8148\2154024332.py:16: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df2_imputed = df2.interpolate()    # <-- Or use median/mean instead


(               count       mean       std    min     25%    50%    75%   max
 temperature_c    8.0  25.075000  0.509201  24.50  24.675  25.00  25.35  26.0
 humidity_pct     9.0  54.966667  0.158114  54.70  54.900  55.00  55.10  55.2
 voltage_v        9.0   3.667778  0.023333   3.63   3.650   3.67   3.68   3.7,
                count     mean       std    min      25%     50%       75%  \
 temperature_c   10.0  25.1100  0.512510  24.50  24.7125  25.000  25.45000   
 humidity_pct    10.0  54.9450  0.164063  54.70  54.8250  54.950  55.07500   
 voltage_v       10.0   3.6695  0.022663   3.63   3.6550   3.675   3.68375   
 
                 max  
 temperature_c  26.0  
 humidity_pct   55.2  
 voltage_v       3.7  )

In this dataset, temperature had the most missing values.
I chose interpolation, because sensor values usually change smoothly over time.
Forward and backward fill work only for short gaps.
Mean and median are too simple and may distort the real patterns.
After imputation, the summary statistics remained realistic, so this method is suitable.